**Part 4: Large Language Model**


In [ ]:
import time, random, numpy as np, argparse, sys, re, os
from types import SimpleNamespace

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, f1_score, recall_score, accuracy_score

# change it with respect to the original model
from tokenizer import BertTokenizer
from bert import BertModel
from tqdm import tqdm
from classifier import BertSentClassifier

***4.1 The fine-tuned minBERT classifier***


Instantiate a BertSentClassifier model. Load the state dict from the checkpoint saved from the
previous question.

Load the best model checkpoint from the previous question. and instantiate a BertSentClassifier

In [ ]:
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
filepath = 'flexible-50-5e-06.pt'

saved = torch.load(filepath)
config = saved['model_config']
model = BertSentClassifier(config)
model.load_state_dict(saved['model'])
model = model.to(device)

Specifiy the batch of sentences

- Very positive: The movie was a masterpiece with a brilliant storyline and exceptional performances by the entire cast.

- Less positive: The film was enjoyable overall, but some of the dialogues felt a bit clichéd.

- Slightly negative: The plot was somewhat predictable, and it failed to capture my interest throughout the movie.

- Very negative: It was a terrible movie with poor acting, a weak script, and subpar visual effects that made it unbearable to watch.

- Off-the-topic: I had a delicious sandwich for lunch from the new café around the corner.

In [ ]:
sentences = [
    "The movie was a masterpiece with a brilliant storyline and exceptional performances by the entire cast.",
    "The film was enjoyable overall, but some of the dialogues felt a bit clichéd.",
    "The plot was somewhat predictable, and it failed to capture my interest throughout the movie.",
    "It was a terrible movie with poor acting, a weak script, and subpar visual effects that made it unbearable to watch.",
    "I had a delicious sandwich for lunch from the new café around the corner."
]

In [ ]:
def pad_data(data):
    sents = [x[0] for x in data]
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    encoding = tokenizer(sents, return_tensors='pt', padding=True, truncation=True)
    token_ids = torch.LongTensor(encoding['input_ids'])
    attention_mask = torch.LongTensor(encoding['attention_mask'])
    token_type_ids = torch.LongTensor(encoding['token_type_ids'])

    return token_ids, token_type_ids, attention_mask, sents


# create the data which is a list of (sentence, label, token for the labels)
def create_data(sentences):
    # specify the tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    data = []

    for line in sentences:
        sent = line.lower().strip()
        tokens = tokenizer.tokenize("[CLS] " + sent + " [SEP]")
        data.append((sent,tokens))
        
    return data

In [ ]:
data = create_data(sentences=sentences)

token_ids, type_ids, mask, sents = pad_data(data)

logits = model(token_ids, mask)
logits = logits.detach().cpu().numpy()

In [ ]:
for i, row in enumerate(logits):
    row = np.exp(row)  # convert to probability from logsofrmax
    positive = row[0] <= row[1]
    print(sentences[i] + '|||' + ('Positive' if positive else 'Negative'))
    print('Positive Probability: ' + str(row[1]))
    print(' Negative Probability: ' + str(row[0]))
    print('------------------------------------')